In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
#Loading dataset
data = pd.read_csv('fer2013.csv')
pixels = data['pixels'].tolist()

# Pre-processing: each pixels array is processed into a numpy array and added to the set
X = []
for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(48, 48)
    face = np.expand_dims(face, axis=-1)
    X.append(face)

X = np.asarray(X)
X = X.astype('float32')
X /= 255.0  # Normlization [0,1]

# Getting the predicted labeles from the set
y = pd.get_dummies(data['emotion']).values  # One-hot encoding

# Divisione in train e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\acaia\anaconda3\envs\first_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
#Fitting the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 53s 116ms/step - accuracy: 0.2561 - loss: 1.8086 - val_accuracy: 0.3911 - val_loss: 1.5647
Epoch 2/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 50s 112ms/step - accuracy: 0.4067 - loss: 1.5273 - val_accuracy: 0.4840 - val_loss: 1.3573
Epoch 3/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 51s 113ms/step - accuracy: 0.4835 - loss: 1.3507 - val_accuracy: 0.5068 - val_loss: 1.3108
Epoch 4/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 51s 114ms/step - accuracy: 0.5230 - loss: 1.2524 - val_accuracy: 0.5337 - val_loss: 1.2205
Epoch 5/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 50s 112ms/step - accuracy: 0.5458 - loss: 1.1900 - val_accuracy: 0.5453 - val_loss: 1.2002
Epoch 6/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 50s 112ms/step - accuracy: 0.5751 - loss: 1.1234 - val_accuracy: 0.5548 - val_loss: 1.1706
Epoch 7/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 52s 115ms/step - accuracy: 0.5966 - loss: 1.0684 - val_accuracy: 0.5620 - val_loss: 1.1553
Epoch 8/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 51s 113ms/step - accuracy: 0.6300 - loss: 0

In [5]:
#Testing
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

#Metrics calculation
accuracy = accuracy_score(y_true_classes, y_pred_classes)
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step
Accuracy: 0.5710504318751741
Precision: 0.5685899640798432
Recall: 0.5710504318751741
F1 Score: 0.5668133686217541


In [6]:
model.save('cnn_FER2013.h5')